In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

In [4]:
# Download training data from open datasets.
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
)

# Download test data from open datasets.
test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)

100.0%


Extracting data\FashionMNIST\raw\train-images-idx3-ubyte.gz to data\FashionMNIST\raw



100.0%


Extracting data\FashionMNIST\raw\train-labels-idx1-ubyte.gz to data\FashionMNIST\raw



100.0%


Extracting data\FashionMNIST\raw\t10k-images-idx3-ubyte.gz to data\FashionMNIST\raw



100.0%

Extracting data\FashionMNIST\raw\t10k-labels-idx1-ubyte.gz to data\FashionMNIST\raw



In [5]:
batch_size = 64

# Create data loaders.
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

for X, y in test_dataloader:
    print(f"Shape of X [N, C, H, W]: {X.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    break

Shape of X [N, C, H, W]: torch.Size([64, 1, 28, 28])
Shape of y: torch.Size([64]) torch.int64


In [6]:
# Get cpu, gpu or mps device for training.
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

# Define model
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10)
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork().to(device)
print(model)

Using cuda device
NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [7]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

In [8]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [9]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [11]:
epochs = 50
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 1.140722  [   64/60000]
loss: 1.149707  [ 6464/60000]
loss: 0.961307  [12864/60000]
loss: 1.105002  [19264/60000]
loss: 0.982036  [25664/60000]
loss: 1.009380  [32064/60000]
loss: 1.060854  [38464/60000]
loss: 0.985263  [44864/60000]
loss: 1.032493  [51264/60000]
loss: 0.952699  [57664/60000]
Test Error: 
 Accuracy: 66.0%, Avg loss: 0.972713 

Epoch 2
-------------------------------
loss: 1.027146  [   64/60000]
loss: 1.057079  [ 6464/60000]
loss: 0.849847  [12864/60000]
loss: 1.015722  [19264/60000]
loss: 0.896640  [25664/60000]
loss: 0.919335  [32064/60000]
loss: 0.987093  [38464/60000]
loss: 0.912945  [44864/60000]
loss: 0.956910  [51264/60000]
loss: 0.888194  [57664/60000]
Test Error: 
 Accuracy: 67.2%, Avg loss: 0.902451 

Epoch 3
-------------------------------
loss: 0.942487  [   64/60000]
loss: 0.991508  [ 6464/60000]
loss: 0.769015  [12864/60000]
loss: 0.951926  [19264/60000]
loss: 0.839234  [25664/60000]
loss: 0.853250  [32064/600

loss: 0.598659  [ 6464/60000]
loss: 0.388412  [12864/60000]
loss: 0.651446  [19264/60000]
loss: 0.573220  [25664/60000]
loss: 0.573699  [32064/60000]
loss: 0.590868  [38464/60000]
loss: 0.672627  [44864/60000]
loss: 0.651980  [51264/60000]
loss: 0.555696  [57664/60000]
Test Error: 
 Accuracy: 80.4%, Avg loss: 0.563186 

Epoch 23
-------------------------------
loss: 0.480785  [   64/60000]
loss: 0.590796  [ 6464/60000]
loss: 0.382185  [12864/60000]
loss: 0.645092  [19264/60000]
loss: 0.567306  [25664/60000]
loss: 0.568983  [32064/60000]
loss: 0.583358  [38464/60000]
loss: 0.672208  [44864/60000]
loss: 0.649793  [51264/60000]
loss: 0.547827  [57664/60000]
Test Error: 
 Accuracy: 80.6%, Avg loss: 0.557476 

Epoch 24
-------------------------------
loss: 0.472881  [   64/60000]
loss: 0.583485  [ 6464/60000]
loss: 0.376348  [12864/60000]
loss: 0.638999  [19264/60000]
loss: 0.561471  [25664/60000]
loss: 0.564292  [32064/60000]
loss: 0.576334  [38464/60000]
loss: 0.672155  [44864/60000]
loss

loss: 0.508674  [ 6464/60000]
loss: 0.310585  [12864/60000]
loss: 0.555231  [19264/60000]
loss: 0.469361  [25664/60000]
loss: 0.487378  [32064/60000]
loss: 0.497464  [38464/60000]
loss: 0.671297  [44864/60000]
loss: 0.616021  [51264/60000]
loss: 0.442681  [57664/60000]
Test Error: 
 Accuracy: 82.5%, Avg loss: 0.492659 

Epoch 44
-------------------------------
loss: 0.366287  [   64/60000]
loss: 0.506493  [ 6464/60000]
loss: 0.308534  [12864/60000]
loss: 0.552069  [19264/60000]
loss: 0.465726  [25664/60000]
loss: 0.484294  [32064/60000]
loss: 0.494825  [38464/60000]
loss: 0.669999  [44864/60000]
loss: 0.613980  [51264/60000]
loss: 0.439761  [57664/60000]
Test Error: 
 Accuracy: 82.6%, Avg loss: 0.490754 

Epoch 45
-------------------------------
loss: 0.362502  [   64/60000]
loss: 0.504347  [ 6464/60000]
loss: 0.306534  [12864/60000]
loss: 0.549034  [19264/60000]
loss: 0.462116  [25664/60000]
loss: 0.481224  [32064/60000]
loss: 0.492265  [38464/60000]
loss: 0.668538  [44864/60000]
loss